path to weights = https://huggingface.co/BIFOLD-BigEarthNetv2-0/resnet50-s1-v0.2.0/blob/main/model.safetensors

Its the version from Sentinel-1 only

In [25]:
import hashlib

BigEarthResnetPthPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/model_S1.safetensors'

# Function to compute SHA-256 hash of a file
def compute_sha256(file_path):
    sha256_hash = hashlib.sha256()
    
    with open(file_path, "rb") as f:
        # Read the file in chunks to handle large files efficiently
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    
    return sha256_hash.hexdigest()

# Compute and print the SHA-256 hash of the file
sha256_hash = compute_sha256(BigEarthResnetPthPath)
print(f"SHA-256: {sha256_hash}")




SHA-256: 87990ba9058bc5413b90d7dc955ed1077cf8c54530db393801336209252b3739


In [26]:
from safetensors import safe_open
name_width = 65
with safe_open(BigEarthResnetPthPath, framework="pt") as f:
    # Print a header
    print("Big Earth Resnet Weights")
    print(f"{'Weight name'.ljust(name_width)} {'Shape'}")
    print("=" * (name_width + 20))  # For a separator line

    # Loop through each tensor
    for key in f.keys():
        tensor = f.get_tensor(key)
        print(f"{key.ljust(name_width)} {str(tensor.shape)}")


Big Earth Resnet Weights
Weight name                                                       Shape
model.vision_encoder.bn1.bias                                     torch.Size([64])
model.vision_encoder.bn1.num_batches_tracked                      torch.Size([])
model.vision_encoder.bn1.running_mean                             torch.Size([64])
model.vision_encoder.bn1.running_var                              torch.Size([64])
model.vision_encoder.bn1.weight                                   torch.Size([64])
model.vision_encoder.conv1.weight                                 torch.Size([64, 2, 7, 7])
model.vision_encoder.fc.bias                                      torch.Size([19])
model.vision_encoder.fc.weight                                    torch.Size([19, 2048])
model.vision_encoder.layer1.0.bn1.bias                            torch.Size([64])
model.vision_encoder.layer1.0.bn1.num_batches_tracked             torch.Size([])
model.vision_encoder.layer1.0.bn1.running_mean                

In [29]:
# SENTINEL 2 Version
BigEarthResnetPthPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/model_S2.safetensors'

sha256_hash = compute_sha256(BigEarthResnetPthPath)
print(f"SHA-256: {sha256_hash}")

name_width = 65
with safe_open(BigEarthResnetPthPath, framework="pt") as f:
    # Print a header
    print("Big Earth Resnet Weights Sentinel 2 (10 bands)")
    print(f"{'Weight name'.ljust(name_width)} {'Shape'}")
    print("=" * (name_width + 20))  # For a separator line

    # Loop through each tensor
    for key in f.keys():
        tensor = f.get_tensor(key)
        print(f"{key.ljust(name_width)} {str(tensor.shape)}")

SHA-256: f55f8a5416d326c7e3139a97c6866e606dfc2e5cde30d52e2ccc47ce5f8b9be5
Big Earth Resnet Weights Sentinel 2 (10 bands)
Weight name                                                       Shape
model.vision_encoder.bn1.bias                                     torch.Size([64])
model.vision_encoder.bn1.num_batches_tracked                      torch.Size([])
model.vision_encoder.bn1.running_mean                             torch.Size([64])
model.vision_encoder.bn1.running_var                              torch.Size([64])
model.vision_encoder.bn1.weight                                   torch.Size([64])
model.vision_encoder.conv1.weight                                 torch.Size([64, 10, 7, 7])
model.vision_encoder.fc.bias                                      torch.Size([19])
model.vision_encoder.fc.weight                                    torch.Size([19, 2048])
model.vision_encoder.layer1.0.bn1.bias                            torch.Size([64])
model.vision_encoder.layer1.0.bn1.num_batches_

In [20]:
import pickle
mask2formerResnetPklPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/R-50.pkl'

# Load the weights from the pickle file
with open(mask2formerResnetPklPath, 'rb') as f:
    weights = pickle.load(f)

# Set the width for alignment
name_width = 65  # Adjust this width based on the longest weight name

# Print header
print("Mask2Former Resnet Weights")
print(f"{'Weight name'.ljust(name_width)} {'Shape'}")
print("=" * (name_width + 20))  # Separator line

# Loop through each weight and print its name and shape
for key, tensor in weights["model"].items():
    print(f"{key.ljust(name_width)} {str(tensor.shape)}")

Mask2Former Resnet Weights
Weight name                                                       Shape
stem.conv1.weight                                                 (64, 3, 7, 7)
stem.conv1.norm.running_mean                                      (64,)
stem.conv1.norm.running_var                                       (64,)
stem.conv1.norm.weight                                            (64,)
stem.conv1.norm.bias                                              (64,)
res2.0.conv1.weight                                               (64, 64, 1, 1)
res2.0.conv1.norm.running_mean                                    (64,)
res2.0.conv1.norm.running_var                                     (64,)
res2.0.conv1.norm.weight                                          (64,)
res2.0.conv1.norm.bias                                            (64,)
res2.0.conv2.weight                                               (64, 64, 3, 3)
res2.0.conv2.norm.running_mean                                    (64,)
res2.0.conv

# Conversion from BigEarth to Mask2former

In [65]:
dictionary_from_Sentinel_to_Mask2former = {
    "model.vision_encoder.fc.weight":"stem.fc.weight ",
    "model.vision_encoder.fc.bias":"stem.fc.bias",

    #STEM
    "model.vision_encoder.conv1.weight":"stem.conv1.weight",
    "model.vision_encoder.bn1.running_mean":"stem.conv1.norm.running_mean",
    "model.vision_encoder.bn1.running_var":"stem.conv1.norm.running_var",
    "model.vision_encoder.bn1.weight":"stem.conv1.norm.weight",
    "model.vision_encoder.bn1.bias":"stem.conv1.norm.bias",
    
    # RES2 Block Layer0
    "model.vision_encoder.layer1.0.conv1.weight":"res2.0.conv1.weight",
    "model.vision_encoder.layer1.0.bn1.running_mean":"res2.0.conv1.norm.running_mean",
    "model.vision_encoder.layer1.0.bn1.running_var":"res2.0.conv1.norm.running_var",
    "model.vision_encoder.layer1.0.bn1.weight":"res2.0.conv1.norm.weight",
    "model.vision_encoder.layer1.0.bn1.bias":"res2.0.conv1.norm.bias",
    "model.vision_encoder.layer1.0.conv2.weight":"res2.0.conv2.weight",
    "model.vision_encoder.layer1.0.bn2.running_mean":"res2.0.conv2.norm.running_mean",
    "model.vision_encoder.layer1.0.bn2.running_var":"res2.0.conv2.norm.running_var",
    "model.vision_encoder.layer1.0.bn2.weight":"res2.0.conv2.norm.weight",
    "model.vision_encoder.layer1.0.bn2.bias":"res2.0.conv2.norm.bias",
    "model.vision_encoder.layer1.0.conv3.weight":"res2.0.conv3.weight",
    "model.vision_encoder.layer1.0.bn3.running_mean":"res2.0.conv3.norm.running_mean",
    "model.vision_encoder.layer1.0.bn3.running_var":"res2.0.conv3.norm.running_var",
    "model.vision_encoder.layer1.0.bn3.weight":"res2.0.conv3.norm.weight",
    "model.vision_encoder.layer1.0.bn3.bias":"res2.0.conv3.norm.bias",
    "model.vision_encoder.layer1.0.downsample.0.weight":"res2.0.shortcut.weight",
    "model.vision_encoder.layer1.0.downsample.1.running_mean":"res2.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer1.0.downsample.1.running_var":"res2.0.shortcut.norm.running_var",
    "model.vision_encoder.layer1.0.downsample.1.weight":"res2.0.shortcut.norm.weight",
    "model.vision_encoder.layer1.0.downsample.1.bias":"res2.0.shortcut.norm.bias",

    # RES2 Block Layer1
    "model.vision_encoder.layer1.1.conv1.weight":"res2.1.conv1.weight",
    "model.vision_encoder.layer1.1.bn1.running_mean":"res2.1.conv1.norm.running_mean",
    "model.vision_encoder.layer1.1.bn1.running_var":"res2.1.conv1.norm.running_var",
    "model.vision_encoder.layer1.1.bn1.weight":"res2.1.conv1.norm.weight",
    "model.vision_encoder.layer1.1.bn1.bias":"res2.1.conv1.norm.bias",
    "model.vision_encoder.layer1.1.conv2.weight":"res2.1.conv2.weight",
    "model.vision_encoder.layer1.1.bn2.running_mean":"res2.1.conv2.norm.running_mean",
    "model.vision_encoder.layer1.1.bn2.running_var":"res2.1.conv2.norm.running_var",
    "model.vision_encoder.layer1.1.bn2.weight":"res2.1.conv2.norm.weight",
    "model.vision_encoder.layer1.1.bn2.bias":"res2.1.conv2.norm.bias",
    "model.vision_encoder.layer1.1.conv3.weight":"res2.1.conv3.weight",
    "model.vision_encoder.layer1.1.bn3.running_mean":"res2.1.conv3.norm.running_mean",
    "model.vision_encoder.layer1.1.bn3.running_var":"res2.1.conv3.norm.running_var",
    "model.vision_encoder.layer1.1.bn3.weight":"res2.1.conv3.norm.weight",
    "model.vision_encoder.layer1.1.bn3.bias":"res2.1.conv3.norm.bias",

    # RES2 Block Layer2
    "model.vision_encoder.layer1.2.conv1.weight":"res2.2.conv1.weight",
    "model.vision_encoder.layer1.2.bn1.running_mean":"res2.2.conv1.norm.running_mean",
    "model.vision_encoder.layer1.2.bn1.running_var":"res2.2.conv1.norm.running_var",
    "model.vision_encoder.layer1.2.bn1.weight":"res2.2.conv1.norm.weight",
    "model.vision_encoder.layer1.2.bn1.bias":"res2.2.conv1.norm.bias",
    "model.vision_encoder.layer1.2.conv2.weight":"res2.2.conv2.weight",
    "model.vision_encoder.layer1.2.bn2.running_mean":"res2.2.conv2.norm.running_mean",
    "model.vision_encoder.layer1.2.bn2.running_var":"res2.2.conv2.norm.running_var",
    "model.vision_encoder.layer1.2.bn2.weight":"res2.2.conv2.norm.weight",
    "model.vision_encoder.layer1.2.bn2.bias":"res2.2.conv2.norm.bias",
    "model.vision_encoder.layer1.2.conv3.weight":"res2.2.conv3.weight",
    "model.vision_encoder.layer1.2.bn3.running_mean":"res2.2.conv3.norm.running_mean",
    "model.vision_encoder.layer1.2.bn3.running_var":"res2.2.conv3.norm.running_var",
    "model.vision_encoder.layer1.2.bn3.weight":"res2.2.conv3.norm.weight",
    "model.vision_encoder.layer1.2.bn3.bias":"res2.2.conv3.norm.bias",

    # RES3 Block Layer0
    "model.vision_encoder.layer2.0.conv1.weight":"res3.0.conv1.weight",
    "model.vision_encoder.layer2.0.bn1.running_mean":"res3.0.conv1.norm.running_mean",
    "model.vision_encoder.layer2.0.bn1.running_var":"res3.0.conv1.norm.running_var",
    "model.vision_encoder.layer2.0.bn1.weight":"res3.0.conv1.norm.weight",
    "model.vision_encoder.layer2.0.bn1.bias":"res3.0.conv1.norm.bias",
    "model.vision_encoder.layer2.0.conv2.weight":"res3.0.conv2.weight",
    "model.vision_encoder.layer2.0.bn2.running_mean":"res3.0.conv2.norm.running_mean",
    "model.vision_encoder.layer2.0.bn2.running_var":"res3.0.conv2.norm.running_var",
    "model.vision_encoder.layer2.0.bn2.weight":"res3.0.conv2.norm.weight",
    "model.vision_encoder.layer2.0.bn2.bias":"res3.0.conv2.norm.bias",
    "model.vision_encoder.layer2.0.conv3.weight":"res3.0.conv3.weight",
    "model.vision_encoder.layer2.0.bn3.running_mean":"res3.0.conv3.norm.running_mean",
    "model.vision_encoder.layer2.0.bn3.running_var":"res3.0.conv3.norm.running_var",
    "model.vision_encoder.layer2.0.bn3.weight":"res3.0.conv3.norm.weight",
    "model.vision_encoder.layer2.0.bn3.bias":"res3.0.conv3.norm.bias",
    "model.vision_encoder.layer2.0.downsample.0.weight":"res3.0.shortcut.weight",
    "model.vision_encoder.layer2.0.downsample.1.running_mean":"res3.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer2.0.downsample.1.running_var":"res3.0.shortcut.norm.running_var",
    "model.vision_encoder.layer2.0.downsample.1.weight":"res3.0.shortcut.norm.weight",
    "model.vision_encoder.layer2.0.downsample.1.bias":"res3.0.shortcut.norm.bias",

    # RES3 Block Layer1
    "model.vision_encoder.layer2.1.conv1.weight":"res3.1.conv1.weight",
    "model.vision_encoder.layer2.1.bn1.running_mean":"res3.1.conv1.norm.running_mean",
    "model.vision_encoder.layer2.1.bn1.running_var":"res3.1.conv1.norm.running_var",
    "model.vision_encoder.layer2.1.bn1.weight":"res3.1.conv1.norm.weight",
    "model.vision_encoder.layer2.1.bn1.bias":"res3.1.conv1.norm.bias",
    "model.vision_encoder.layer2.1.conv2.weight":"res3.1.conv2.weight",
    "model.vision_encoder.layer2.1.bn2.running_mean":"res3.1.conv2.norm.running_mean",
    "model.vision_encoder.layer2.1.bn2.running_var":"res3.1.conv2.norm.running_var",
    "model.vision_encoder.layer2.1.bn2.weight":"res3.1.conv2.norm.weight",
    "model.vision_encoder.layer2.1.bn2.bias":"res3.1.conv2.norm.bias",
    "model.vision_encoder.layer2.1.conv3.weight":"res3.1.conv3.weight",
    "model.vision_encoder.layer2.1.bn3.running_mean":"res3.1.conv3.norm.running_mean",
    "model.vision_encoder.layer2.1.bn3.running_var":"res3.1.conv3.norm.running_var",
    "model.vision_encoder.layer2.1.bn3.weight":"res3.1.conv3.norm.weight",
    "model.vision_encoder.layer2.1.bn3.bias":"res3.1.conv3.norm.bias",

    # RES3 Block Layer2
    "model.vision_encoder.layer2.2.conv1.weight":"res3.2.conv1.weight",
    "model.vision_encoder.layer2.2.bn1.running_mean":"res3.2.conv1.norm.running_mean",
    "model.vision_encoder.layer2.2.bn1.running_var":"res3.2.conv1.norm.running_var",
    "model.vision_encoder.layer2.2.bn1.weight":"res3.2.conv1.norm.weight",
    "model.vision_encoder.layer2.2.bn1.bias":"res3.2.conv1.norm.bias",
    "model.vision_encoder.layer2.2.conv2.weight":"res3.2.conv2.weight",
    "model.vision_encoder.layer2.2.bn2.running_mean":"res3.2.conv2.norm.running_mean",
    "model.vision_encoder.layer2.2.bn2.running_var":"res3.2.conv2.norm.running_var",
    "model.vision_encoder.layer2.2.bn2.weight":"res3.2.conv2.norm.weight",
    "model.vision_encoder.layer2.2.bn2.bias":"res3.2.conv2.norm.bias",
    "model.vision_encoder.layer2.2.conv3.weight":"res3.2.conv3.weight",
    "model.vision_encoder.layer2.2.bn3.running_mean":"res3.2.conv3.norm.running_mean",
    "model.vision_encoder.layer2.2.bn3.running_var":"res3.2.conv3.norm.running_var",
    "model.vision_encoder.layer2.2.bn3.weight":"res3.2.conv3.norm.weight",
    "model.vision_encoder.layer2.2.bn3.bias":"res3.2.conv3.norm.bias",

    # RES3 Block Layer3
    "model.vision_encoder.layer2.3.conv1.weight":"res3.3.conv1.weight",
    "model.vision_encoder.layer2.3.bn1.running_mean":"res3.3.conv1.norm.running_mean",
    "model.vision_encoder.layer2.3.bn1.running_var":"res3.3.conv1.norm.running_var",
    "model.vision_encoder.layer2.3.bn1.weight":"res3.3.conv1.norm.weight",
    "model.vision_encoder.layer2.3.bn1.bias":"res3.3.conv1.norm.bias",
    "model.vision_encoder.layer2.3.conv2.weight":"res3.3.conv2.weight",
    "model.vision_encoder.layer2.3.bn2.running_mean":"res3.3.conv2.norm.running_mean",
    "model.vision_encoder.layer2.3.bn2.running_var":"res3.3.conv2.norm.running_var",
    "model.vision_encoder.layer2.3.bn2.weight":"res3.3.conv2.norm.weight",
    "model.vision_encoder.layer2.3.bn2.bias":"res3.3.conv2.norm.bias",
    "model.vision_encoder.layer2.3.conv3.weight":"res3.3.conv3.weight",
    "model.vision_encoder.layer2.3.bn3.running_mean":"res3.3.conv3.norm.running_mean",
    "model.vision_encoder.layer2.3.bn3.running_var":"res3.3.conv3.norm.running_var",
    "model.vision_encoder.layer2.3.bn3.weight":"res3.3.conv3.norm.weight",
    "model.vision_encoder.layer2.3.bn3.bias":"res3.3.conv3.norm.bias",

    # RES4 Block Layer0
    "model.vision_encoder.layer3.0.conv1.weight":"res4.0.conv1.weight",
    "model.vision_encoder.layer3.0.bn1.running_mean":"res4.0.conv1.norm.running_mean",
    "model.vision_encoder.layer3.0.bn1.running_var":"res4.0.conv1.norm.running_var",
    "model.vision_encoder.layer3.0.bn1.weight":"res4.0.conv1.norm.weight",
    "model.vision_encoder.layer3.0.bn1.bias":"res4.0.conv1.norm.bias",
    "model.vision_encoder.layer3.0.conv2.weight":"res4.0.conv2.weight",
    "model.vision_encoder.layer3.0.bn2.running_mean":"res4.0.conv2.norm.running_mean",
    "model.vision_encoder.layer3.0.bn2.running_var":"res4.0.conv2.norm.running_var",
    "model.vision_encoder.layer3.0.bn2.weight":"res4.0.conv2.norm.weight",
    "model.vision_encoder.layer3.0.bn2.bias":"res4.0.conv2.norm.bias",
    "model.vision_encoder.layer3.0.conv3.weight":"res4.0.conv3.weight",
    "model.vision_encoder.layer3.0.bn3.running_mean":"res4.0.conv3.norm.running_mean",
    "model.vision_encoder.layer3.0.bn3.running_var":"res4.0.conv3.norm.running_var",
    "model.vision_encoder.layer3.0.bn3.weight":"res4.0.conv3.norm.weight",
    "model.vision_encoder.layer3.0.bn3.bias":"res4.0.conv3.norm.bias",
    "model.vision_encoder.layer3.0.downsample.0.weight":"res4.0.shortcut.weight",
    "model.vision_encoder.layer3.0.downsample.1.running_mean":"res4.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer3.0.downsample.1.running_var":"res4.0.shortcut.norm.running_var",
    "model.vision_encoder.layer3.0.downsample.1.weight":"res4.0.shortcut.norm.weight",
    "model.vision_encoder.layer3.0.downsample.1.bias":"res4.0.shortcut.norm.bias",

    # RES4 Block Layer1
    "model.vision_encoder.layer3.1.conv1.weight":"res4.1.conv1.weight",
    "model.vision_encoder.layer3.1.bn1.running_mean":"res4.1.conv1.norm.running_mean",
    "model.vision_encoder.layer3.1.bn1.running_var":"res4.1.conv1.norm.running_var",
    "model.vision_encoder.layer3.1.bn1.weight":"res4.1.conv1.norm.weight",
    "model.vision_encoder.layer3.1.bn1.bias":"res4.1.conv1.norm.bias",
    "model.vision_encoder.layer3.1.conv2.weight":"res4.1.conv2.weight",
    "model.vision_encoder.layer3.1.bn2.running_mean":"res4.1.conv2.norm.running_mean",
    "model.vision_encoder.layer3.1.bn2.running_var":"res4.1.conv2.norm.running_var",
    "model.vision_encoder.layer3.1.bn2.weight":"res4.1.conv2.norm.weight",
    "model.vision_encoder.layer3.1.bn2.bias":"res4.1.conv2.norm.bias",
    "model.vision_encoder.layer3.1.conv3.weight":"res4.1.conv3.weight",
    "model.vision_encoder.layer3.1.bn3.running_mean":"res4.1.conv3.norm.running_mean",
    "model.vision_encoder.layer3.1.bn3.running_var":"res4.1.conv3.norm.running_var",
    "model.vision_encoder.layer3.1.bn3.weight":"res4.1.conv3.norm.weight",
    "model.vision_encoder.layer3.1.bn3.bias":"res4.1.conv3.norm.bias",

    # RES4 Block Layer2
    "model.vision_encoder.layer3.2.conv1.weight":"res4.2.conv1.weight",
    "model.vision_encoder.layer3.2.bn1.running_mean":"res4.2.conv1.norm.running_mean",
    "model.vision_encoder.layer3.2.bn1.running_var":"res4.2.conv1.norm.running_var",
    "model.vision_encoder.layer3.2.bn1.weight":"res4.2.conv1.norm.weight",
    "model.vision_encoder.layer3.2.bn1.bias":"res4.2.conv1.norm.bias",
    "model.vision_encoder.layer3.2.conv2.weight":"res4.2.conv2.weight",
    "model.vision_encoder.layer3.2.bn2.running_mean":"res4.2.conv2.norm.running_mean",
    "model.vision_encoder.layer3.2.bn2.running_var":"res4.2.conv2.norm.running_var",
    "model.vision_encoder.layer3.2.bn2.weight":"res4.2.conv2.norm.weight",
    "model.vision_encoder.layer3.2.bn2.bias":"res4.2.conv2.norm.bias",
    "model.vision_encoder.layer3.2.conv3.weight":"res4.2.conv3.weight",
    "model.vision_encoder.layer3.2.bn3.running_mean":"res4.2.conv3.norm.running_mean",
    "model.vision_encoder.layer3.2.bn3.running_var":"res4.2.conv3.norm.running_var",
    "model.vision_encoder.layer3.2.bn3.weight":"res4.2.conv3.norm.weight",
    "model.vision_encoder.layer3.2.bn3.bias":"res4.2.conv3.norm.bias",

    # RES4 Block Layer3
    "model.vision_encoder.layer3.3.conv1.weight":"res4.3.conv1.weight",
    "model.vision_encoder.layer3.3.bn1.running_mean":"res4.3.conv1.norm.running_mean",
    "model.vision_encoder.layer3.3.bn1.running_var":"res4.3.conv1.norm.running_var",
    "model.vision_encoder.layer3.3.bn1.weight":"res4.3.conv1.norm.weight",
    "model.vision_encoder.layer3.3.bn1.bias":"res4.3.conv1.norm.bias",
    "model.vision_encoder.layer3.3.conv2.weight":"res4.3.conv2.weight",
    "model.vision_encoder.layer3.3.bn2.running_mean":"res4.3.conv2.norm.running_mean",
    "model.vision_encoder.layer3.3.bn2.running_var":"res4.3.conv2.norm.running_var",
    "model.vision_encoder.layer3.3.bn2.weight":"res4.3.conv2.norm.weight",
    "model.vision_encoder.layer3.3.bn2.bias":"res4.3.conv2.norm.bias",
    "model.vision_encoder.layer3.3.conv3.weight":"res4.3.conv3.weight",
    "model.vision_encoder.layer3.3.bn3.running_mean":"res4.3.conv3.norm.running_mean",
    "model.vision_encoder.layer3.3.bn3.running_var":"res4.3.conv3.norm.running_var",
    "model.vision_encoder.layer3.3.bn3.weight":"res4.3.conv3.norm.weight",
    "model.vision_encoder.layer3.3.bn3.bias":"res4.3.conv3.norm.bias",

    # RES4 Block Layer4
    "model.vision_encoder.layer3.4.conv1.weight":"res4.4.conv1.weight",
    "model.vision_encoder.layer3.4.bn1.running_mean":"res4.4.conv1.norm.running_mean",
    "model.vision_encoder.layer3.4.bn1.running_var":"res4.4.conv1.norm.running_var",
    "model.vision_encoder.layer3.4.bn1.weight":"res4.4.conv1.norm.weight",
    "model.vision_encoder.layer3.4.bn1.bias":"res4.4.conv1.norm.bias",
    "model.vision_encoder.layer3.4.conv2.weight":"res4.4.conv2.weight",
    "model.vision_encoder.layer3.4.bn2.running_mean":"res4.4.conv2.norm.running_mean",
    "model.vision_encoder.layer3.4.bn2.running_var":"res4.4.conv2.norm.running_var",
    "model.vision_encoder.layer3.4.bn2.weight":"res4.4.conv2.norm.weight",
    "model.vision_encoder.layer3.4.bn2.bias":"res4.4.conv2.norm.bias",
    "model.vision_encoder.layer3.4.conv3.weight":"res4.4.conv3.weight",
    "model.vision_encoder.layer3.4.bn3.running_mean":"res4.4.conv3.norm.running_mean",
    "model.vision_encoder.layer3.4.bn3.running_var":"res4.4.conv3.norm.running_var",
    "model.vision_encoder.layer3.4.bn3.weight":"res4.4.conv3.norm.weight",
    "model.vision_encoder.layer3.4.bn3.bias":"res4.4.conv3.norm.bias",

    # RES4 Block Layer5
    "model.vision_encoder.layer3.5.conv1.weight":"res4.5.conv1.weight",
    "model.vision_encoder.layer3.5.bn1.running_mean":"res4.5.conv1.norm.running_mean",
    "model.vision_encoder.layer3.5.bn1.running_var":"res4.5.conv1.norm.running_var",
    "model.vision_encoder.layer3.5.bn1.weight":"res4.5.conv1.norm.weight",
    "model.vision_encoder.layer3.5.bn1.bias":"res4.5.conv1.norm.bias",
    "model.vision_encoder.layer3.5.conv2.weight":"res4.5.conv2.weight",
    "model.vision_encoder.layer3.5.bn2.running_mean":"res4.5.conv2.norm.running_mean",
    "model.vision_encoder.layer3.5.bn2.running_var":"res4.5.conv2.norm.running_var",
    "model.vision_encoder.layer3.5.bn2.weight":"res4.5.conv2.norm.weight",
    "model.vision_encoder.layer3.5.bn2.bias":"res4.5.conv2.norm.bias",
    "model.vision_encoder.layer3.5.conv3.weight":"res4.5.conv3.weight",
    "model.vision_encoder.layer3.5.bn3.running_mean":"res4.5.conv3.norm.running_mean",
    "model.vision_encoder.layer3.5.bn3.running_var":"res4.5.conv3.norm.running_var",
    "model.vision_encoder.layer3.5.bn3.weight":"res4.5.conv3.norm.weight",
    "model.vision_encoder.layer3.5.bn3.bias":"res4.5.conv3.norm.bias",

    # RES5 Block Layer0
    "model.vision_encoder.layer4.0.conv1.weight":"res5.0.conv1.weight",
    "model.vision_encoder.layer4.0.bn1.running_mean":"res5.0.conv1.norm.running_mean",
    "model.vision_encoder.layer4.0.bn1.running_var":"res5.0.conv1.norm.running_var",
    "model.vision_encoder.layer4.0.bn1.weight":"res5.0.conv1.norm.weight",
    "model.vision_encoder.layer4.0.bn1.bias":"res5.0.conv1.norm.bias",
    "model.vision_encoder.layer4.0.conv2.weight":"res5.0.conv2.weight",
    "model.vision_encoder.layer4.0.bn2.running_mean":"res5.0.conv2.norm.running_mean",
    "model.vision_encoder.layer4.0.bn2.running_var":"res5.0.conv2.norm.running_var",
    "model.vision_encoder.layer4.0.bn2.weight":"res5.0.conv2.norm.weight",
    "model.vision_encoder.layer4.0.bn2.bias":"res5.0.conv2.norm.bias",
    "model.vision_encoder.layer4.0.conv3.weight":"res5.0.conv3.weight",
    "model.vision_encoder.layer4.0.bn3.running_mean":"res5.0.conv3.norm.running_mean",
    "model.vision_encoder.layer4.0.bn3.running_var":"res5.0.conv3.norm.running_var",
    "model.vision_encoder.layer4.0.bn3.weight":"res5.0.conv3.norm.weight",
    "model.vision_encoder.layer4.0.bn3.bias":"res5.0.conv3.norm.bias",
    "model.vision_encoder.layer4.0.downsample.0.weight":"res5.0.shortcut.weight",
    "model.vision_encoder.layer4.0.downsample.1.running_mean":"res5.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer4.0.downsample.1.running_var":"res5.0.shortcut.norm.running_var",
    "model.vision_encoder.layer4.0.downsample.1.weight":"res5.0.shortcut.norm.weight",
    "model.vision_encoder.layer4.0.downsample.1.bias":"res5.0.shortcut.norm.bias",

    # RES5 Block Layer1
    "model.vision_encoder.layer4.1.conv1.weight":"res5.1.conv1.weight",
    "model.vision_encoder.layer4.1.bn1.running_mean":"res5.1.conv1.norm.running_mean",
    "model.vision_encoder.layer4.1.bn1.running_var":"res5.1.conv1.norm.running_var",
    "model.vision_encoder.layer4.1.bn1.weight":"res5.1.conv1.norm.weight",
    "model.vision_encoder.layer4.1.bn1.bias":"res5.1.conv1.norm.bias",
    "model.vision_encoder.layer4.1.conv2.weight":"res5.1.conv2.weight",
    "model.vision_encoder.layer4.1.bn2.running_mean":"res5.1.conv2.norm.running_mean",
    "model.vision_encoder.layer4.1.bn2.running_var":"res5.1.conv2.norm.running_var",
    "model.vision_encoder.layer4.1.bn2.weight":"res5.1.conv2.norm.weight",
    "model.vision_encoder.layer4.1.bn2.bias":"res5.1.conv2.norm.bias",
    "model.vision_encoder.layer4.1.conv3.weight":"res5.1.conv3.weight",
    "model.vision_encoder.layer4.1.bn3.running_mean":"res5.1.conv3.norm.running_mean",
    "model.vision_encoder.layer4.1.bn3.running_var":"res5.1.conv3.norm.running_var",
    "model.vision_encoder.layer4.1.bn3.weight":"res5.1.conv3.norm.weight",
    "model.vision_encoder.layer4.1.bn3.bias":"res5.1.conv3.norm.bias",

    # RES5 Block Layer2
    "model.vision_encoder.layer4.2.conv1.weight":"res5.2.conv1.weight",
    "model.vision_encoder.layer4.2.bn1.running_mean":"res5.2.conv1.norm.running_mean",
    "model.vision_encoder.layer4.2.bn1.running_var":"res5.2.conv1.norm.running_var",
    "model.vision_encoder.layer4.2.bn1.weight":"res5.2.conv1.norm.weight",
    "model.vision_encoder.layer4.2.bn1.bias":"res5.2.conv1.norm.bias",
    "model.vision_encoder.layer4.2.conv2.weight":"res5.2.conv2.weight",
    "model.vision_encoder.layer4.2.bn2.running_mean":"res5.2.conv2.norm.running_mean",
    "model.vision_encoder.layer4.2.bn2.running_var":"res5.2.conv2.norm.running_var",
    "model.vision_encoder.layer4.2.bn2.weight":"res5.2.conv2.norm.weight",
    "model.vision_encoder.layer4.2.bn2.bias":"res5.2.conv2.norm.bias",
    "model.vision_encoder.layer4.2.conv3.weight":"res5.2.conv3.weight",
    "model.vision_encoder.layer4.2.bn3.running_mean":"res5.2.conv3.norm.running_mean",
    "model.vision_encoder.layer4.2.bn3.running_var":"res5.2.conv3.norm.running_var",
    "model.vision_encoder.layer4.2.bn3.weight":"res5.2.conv3.norm.weight",
    "model.vision_encoder.layer4.2.bn3.bias":"res5.2.conv3.norm.bias",
}

In [66]:
import numpy as np
mask2formerResnetPklPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/R-50.pkl'
BigEarthResnetPthPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/model_S2.safetensors'

# Keep the Bigearth weights in a dictionary with the keys from the Mask2Former model
bigearth_weights = {}
with safe_open(BigEarthResnetPthPath, framework="pt") as f:
    for key in f.keys():
        if key in dictionary_from_Sentinel_to_Mask2former:
            new_key = dictionary_from_Sentinel_to_Mask2former[key]
            bigearth_weights[new_key] = np.array(f.get_tensor(key))

for key in bigearth_weights.keys():
    print(key)
    print(bigearth_weights[key].shape)

# Load the weights from the pickle file
with open(mask2formerResnetPklPath, 'rb') as f:
    weights = pickle.load(f)

weights["model"] = bigearth_weights

# Save the updated weights to a new pickle file
new_pkl_path = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/R-50_S2.pkl'
with open(new_pkl_path, 'wb') as f:
    pickle.dump(weights, f)


stem.conv1.norm.bias
(64,)
stem.conv1.norm.running_mean
(64,)
stem.conv1.norm.running_var
(64,)
stem.conv1.norm.weight
(64,)
stem.conv1.weight
(64, 10, 7, 7)
stem.fc.bias
(19,)
stem.fc.weight 
(19, 2048)
res2.0.conv1.norm.bias
(64,)
res2.0.conv1.norm.running_mean
(64,)
res2.0.conv1.norm.running_var
(64,)
res2.0.conv1.norm.weight
(64,)
res2.0.conv2.norm.bias
(64,)
res2.0.conv2.norm.running_mean
(64,)
res2.0.conv2.norm.running_var
(64,)
res2.0.conv2.norm.weight
(64,)
res2.0.conv3.norm.bias
(256,)
res2.0.conv3.norm.running_mean
(256,)
res2.0.conv3.norm.running_var
(256,)
res2.0.conv3.norm.weight
(256,)
res2.0.conv1.weight
(64, 64, 1, 1)
res2.0.conv2.weight
(64, 64, 3, 3)
res2.0.conv3.weight
(256, 64, 1, 1)
res2.0.shortcut.weight
(256, 64, 1, 1)
res2.0.shortcut.norm.bias
(256,)
res2.0.shortcut.norm.running_mean
(256,)
res2.0.shortcut.norm.running_var
(256,)
res2.0.shortcut.norm.weight
(256,)
res2.1.conv1.norm.bias
(64,)
res2.1.conv1.norm.running_mean
(64,)
res2.1.conv1.norm.running_var
(64

In [67]:
# load the new pickle file
with open(new_pkl_path, 'rb') as f:
    weights = pickle.load(f)

print(weights.keys())
print(weights["model"].keys())

for key in weights["model"].keys():
    print(key)
    print(weights["model"][key].shape)


dict_keys(['model', '__author__', 'matching_heuristics'])
dict_keys(['stem.conv1.norm.bias', 'stem.conv1.norm.running_mean', 'stem.conv1.norm.running_var', 'stem.conv1.norm.weight', 'stem.conv1.weight', 'stem.fc.bias', 'stem.fc.weight ', 'res2.0.conv1.norm.bias', 'res2.0.conv1.norm.running_mean', 'res2.0.conv1.norm.running_var', 'res2.0.conv1.norm.weight', 'res2.0.conv2.norm.bias', 'res2.0.conv2.norm.running_mean', 'res2.0.conv2.norm.running_var', 'res2.0.conv2.norm.weight', 'res2.0.conv3.norm.bias', 'res2.0.conv3.norm.running_mean', 'res2.0.conv3.norm.running_var', 'res2.0.conv3.norm.weight', 'res2.0.conv1.weight', 'res2.0.conv2.weight', 'res2.0.conv3.weight', 'res2.0.shortcut.weight', 'res2.0.shortcut.norm.bias', 'res2.0.shortcut.norm.running_mean', 'res2.0.shortcut.norm.running_var', 'res2.0.shortcut.norm.weight', 'res2.1.conv1.norm.bias', 'res2.1.conv1.norm.running_mean', 'res2.1.conv1.norm.running_var', 'res2.1.conv1.norm.weight', 'res2.1.conv2.norm.bias', 'res2.1.conv2.norm.runni

Según la traza del entrenamiento le faltan estos pesos

backbone.res2.0.conv3.norm.bias
backbone.res2.1.conv3.norm.bias
backbone.res2.2.conv3.norm.bias
backbone.res3.0.conv3.norm.bias
backbone.res3.1.conv3.norm.bias
backbone.res3.2.conv3.norm.bias
backbone.res3.3.conv3.norm.bias
backbone.res4.0.conv3.norm.bias
backbone.res4.1.conv3.norm.bias
backbone.res4.2.conv3.norm.bias
backbone.res4.3.conv3.norm.bias
backbone.res4.4.conv3.norm.bias
backbone.res4.5.conv3.norm.bias
backbone.res5.0.conv3.norm.bias
backbone.res5.1.conv3.norm.bias
backbone.res5.2.conv3.norm.bias
backbone.stem.conv1.norm.bias
backbone.stem.conv1.weight

criterion.empty_weight


WARNING [10/25 09:18:33 fvcore.common.checkpoint]: The checkpoint state_dict contains keys that are not used by the model:
  stem.conv1.weight
  stem.fc.{bias, weight }

In [68]:
import numpy as np
mask2formerResnetPklPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/R-50.pkl'
BigEarthResnetPthPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/model_s1+2.safetensors'

# Keep the Bigearth weights in a dictionary with the keys from the Mask2Former model
bigearth_weights = {}
with safe_open(BigEarthResnetPthPath, framework="pt") as f:
    for key in f.keys():
        if key in dictionary_from_Sentinel_to_Mask2former:
            new_key = dictionary_from_Sentinel_to_Mask2former[key]
            bigearth_weights[new_key] = np.array(f.get_tensor(key))

for key in bigearth_weights.keys():
    print(key)
    print(bigearth_weights[key].shape)

# Load the weights from the pickle file
with open(mask2formerResnetPklPath, 'rb') as f:
    weights = pickle.load(f)

weights["model"] = bigearth_weights

# Save the updated weights to a new pickle file
new_pkl_path = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/R-50_S2+1.pkl'
with open(new_pkl_path, 'wb') as f:
    pickle.dump(weights, f)

stem.conv1.norm.bias
(64,)
stem.conv1.norm.running_mean
(64,)
stem.conv1.norm.running_var
(64,)
stem.conv1.norm.weight
(64,)
stem.conv1.weight
(64, 12, 7, 7)
stem.fc.bias
(19,)
stem.fc.weight 
(19, 2048)
res2.0.conv1.norm.bias
(64,)
res2.0.conv1.norm.running_mean
(64,)
res2.0.conv1.norm.running_var
(64,)
res2.0.conv1.norm.weight
(64,)
res2.0.conv2.norm.bias
(64,)
res2.0.conv2.norm.running_mean
(64,)
res2.0.conv2.norm.running_var
(64,)
res2.0.conv2.norm.weight
(64,)
res2.0.conv3.norm.bias
(256,)
res2.0.conv3.norm.running_mean
(256,)
res2.0.conv3.norm.running_var
(256,)
res2.0.conv3.norm.weight
(256,)
res2.0.conv1.weight
(64, 64, 1, 1)
res2.0.conv2.weight
(64, 64, 3, 3)
res2.0.conv3.weight
(256, 64, 1, 1)
res2.0.shortcut.weight
(256, 64, 1, 1)
res2.0.shortcut.norm.bias
(256,)
res2.0.shortcut.norm.running_mean
(256,)
res2.0.shortcut.norm.running_var
(256,)
res2.0.shortcut.norm.weight
(256,)
res2.1.conv1.norm.bias
(64,)
res2.1.conv1.norm.running_mean
(64,)
res2.1.conv1.norm.running_var
(64

# Correct Order of bands for the first layer

In [1]:
dictionary_from_Sentinel_to_Mask2former = {
    "model.vision_encoder.fc.weight":"stem.fc.weight ",
    "model.vision_encoder.fc.bias":"stem.fc.bias",

    #STEM
    #"model.vision_encoder.conv1.weight":"stem.conv1.weight", # Esta es la que tengo que ajustar a mano # el shape que tiene es (64, 8, 7, 7), tengo que convertirlo a (64, 4, 7, 7) teniendo en cuenta el orden correcto de bandas
    "model.vision_encoder.bn1.running_mean":"stem.conv1.norm.running_mean",
    "model.vision_encoder.bn1.running_var":"stem.conv1.norm.running_var",
    "model.vision_encoder.bn1.weight":"stem.conv1.norm.weight",
    "model.vision_encoder.bn1.bias":"stem.conv1.norm.bias",
    
    # RES2 Block Layer0
    "model.vision_encoder.layer1.0.conv1.weight":"res2.0.conv1.weight",
    "model.vision_encoder.layer1.0.bn1.running_mean":"res2.0.conv1.norm.running_mean",
    "model.vision_encoder.layer1.0.bn1.running_var":"res2.0.conv1.norm.running_var",
    "model.vision_encoder.layer1.0.bn1.weight":"res2.0.conv1.norm.weight",
    "model.vision_encoder.layer1.0.bn1.bias":"res2.0.conv1.norm.bias",
    "model.vision_encoder.layer1.0.conv2.weight":"res2.0.conv2.weight",
    "model.vision_encoder.layer1.0.bn2.running_mean":"res2.0.conv2.norm.running_mean",
    "model.vision_encoder.layer1.0.bn2.running_var":"res2.0.conv2.norm.running_var",
    "model.vision_encoder.layer1.0.bn2.weight":"res2.0.conv2.norm.weight",
    "model.vision_encoder.layer1.0.bn2.bias":"res2.0.conv2.norm.bias",
    "model.vision_encoder.layer1.0.conv3.weight":"res2.0.conv3.weight",
    "model.vision_encoder.layer1.0.bn3.running_mean":"res2.0.conv3.norm.running_mean",
    "model.vision_encoder.layer1.0.bn3.running_var":"res2.0.conv3.norm.running_var",
    "model.vision_encoder.layer1.0.bn3.weight":"res2.0.conv3.norm.weight",
    "model.vision_encoder.layer1.0.bn3.bias":"res2.0.conv3.norm.bias",
    "model.vision_encoder.layer1.0.downsample.0.weight":"res2.0.shortcut.weight",
    "model.vision_encoder.layer1.0.downsample.1.running_mean":"res2.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer1.0.downsample.1.running_var":"res2.0.shortcut.norm.running_var",
    "model.vision_encoder.layer1.0.downsample.1.weight":"res2.0.shortcut.norm.weight",
    "model.vision_encoder.layer1.0.downsample.1.bias":"res2.0.shortcut.norm.bias",

    # RES2 Block Layer1
    "model.vision_encoder.layer1.1.conv1.weight":"res2.1.conv1.weight",
    "model.vision_encoder.layer1.1.bn1.running_mean":"res2.1.conv1.norm.running_mean",
    "model.vision_encoder.layer1.1.bn1.running_var":"res2.1.conv1.norm.running_var",
    "model.vision_encoder.layer1.1.bn1.weight":"res2.1.conv1.norm.weight",
    "model.vision_encoder.layer1.1.bn1.bias":"res2.1.conv1.norm.bias",
    "model.vision_encoder.layer1.1.conv2.weight":"res2.1.conv2.weight",
    "model.vision_encoder.layer1.1.bn2.running_mean":"res2.1.conv2.norm.running_mean",
    "model.vision_encoder.layer1.1.bn2.running_var":"res2.1.conv2.norm.running_var",
    "model.vision_encoder.layer1.1.bn2.weight":"res2.1.conv2.norm.weight",
    "model.vision_encoder.layer1.1.bn2.bias":"res2.1.conv2.norm.bias",
    "model.vision_encoder.layer1.1.conv3.weight":"res2.1.conv3.weight",
    "model.vision_encoder.layer1.1.bn3.running_mean":"res2.1.conv3.norm.running_mean",
    "model.vision_encoder.layer1.1.bn3.running_var":"res2.1.conv3.norm.running_var",
    "model.vision_encoder.layer1.1.bn3.weight":"res2.1.conv3.norm.weight",
    "model.vision_encoder.layer1.1.bn3.bias":"res2.1.conv3.norm.bias",

    # RES2 Block Layer2
    "model.vision_encoder.layer1.2.conv1.weight":"res2.2.conv1.weight",
    "model.vision_encoder.layer1.2.bn1.running_mean":"res2.2.conv1.norm.running_mean",
    "model.vision_encoder.layer1.2.bn1.running_var":"res2.2.conv1.norm.running_var",
    "model.vision_encoder.layer1.2.bn1.weight":"res2.2.conv1.norm.weight",
    "model.vision_encoder.layer1.2.bn1.bias":"res2.2.conv1.norm.bias",
    "model.vision_encoder.layer1.2.conv2.weight":"res2.2.conv2.weight",
    "model.vision_encoder.layer1.2.bn2.running_mean":"res2.2.conv2.norm.running_mean",
    "model.vision_encoder.layer1.2.bn2.running_var":"res2.2.conv2.norm.running_var",
    "model.vision_encoder.layer1.2.bn2.weight":"res2.2.conv2.norm.weight",
    "model.vision_encoder.layer1.2.bn2.bias":"res2.2.conv2.norm.bias",
    "model.vision_encoder.layer1.2.conv3.weight":"res2.2.conv3.weight",
    "model.vision_encoder.layer1.2.bn3.running_mean":"res2.2.conv3.norm.running_mean",
    "model.vision_encoder.layer1.2.bn3.running_var":"res2.2.conv3.norm.running_var",
    "model.vision_encoder.layer1.2.bn3.weight":"res2.2.conv3.norm.weight",
    "model.vision_encoder.layer1.2.bn3.bias":"res2.2.conv3.norm.bias",

    # RES3 Block Layer0
    "model.vision_encoder.layer2.0.conv1.weight":"res3.0.conv1.weight",
    "model.vision_encoder.layer2.0.bn1.running_mean":"res3.0.conv1.norm.running_mean",
    "model.vision_encoder.layer2.0.bn1.running_var":"res3.0.conv1.norm.running_var",
    "model.vision_encoder.layer2.0.bn1.weight":"res3.0.conv1.norm.weight",
    "model.vision_encoder.layer2.0.bn1.bias":"res3.0.conv1.norm.bias",
    "model.vision_encoder.layer2.0.conv2.weight":"res3.0.conv2.weight",
    "model.vision_encoder.layer2.0.bn2.running_mean":"res3.0.conv2.norm.running_mean",
    "model.vision_encoder.layer2.0.bn2.running_var":"res3.0.conv2.norm.running_var",
    "model.vision_encoder.layer2.0.bn2.weight":"res3.0.conv2.norm.weight",
    "model.vision_encoder.layer2.0.bn2.bias":"res3.0.conv2.norm.bias",
    "model.vision_encoder.layer2.0.conv3.weight":"res3.0.conv3.weight",
    "model.vision_encoder.layer2.0.bn3.running_mean":"res3.0.conv3.norm.running_mean",
    "model.vision_encoder.layer2.0.bn3.running_var":"res3.0.conv3.norm.running_var",
    "model.vision_encoder.layer2.0.bn3.weight":"res3.0.conv3.norm.weight",
    "model.vision_encoder.layer2.0.bn3.bias":"res3.0.conv3.norm.bias",
    "model.vision_encoder.layer2.0.downsample.0.weight":"res3.0.shortcut.weight",
    "model.vision_encoder.layer2.0.downsample.1.running_mean":"res3.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer2.0.downsample.1.running_var":"res3.0.shortcut.norm.running_var",
    "model.vision_encoder.layer2.0.downsample.1.weight":"res3.0.shortcut.norm.weight",
    "model.vision_encoder.layer2.0.downsample.1.bias":"res3.0.shortcut.norm.bias",

    # RES3 Block Layer1
    "model.vision_encoder.layer2.1.conv1.weight":"res3.1.conv1.weight",
    "model.vision_encoder.layer2.1.bn1.running_mean":"res3.1.conv1.norm.running_mean",
    "model.vision_encoder.layer2.1.bn1.running_var":"res3.1.conv1.norm.running_var",
    "model.vision_encoder.layer2.1.bn1.weight":"res3.1.conv1.norm.weight",
    "model.vision_encoder.layer2.1.bn1.bias":"res3.1.conv1.norm.bias",
    "model.vision_encoder.layer2.1.conv2.weight":"res3.1.conv2.weight",
    "model.vision_encoder.layer2.1.bn2.running_mean":"res3.1.conv2.norm.running_mean",
    "model.vision_encoder.layer2.1.bn2.running_var":"res3.1.conv2.norm.running_var",
    "model.vision_encoder.layer2.1.bn2.weight":"res3.1.conv2.norm.weight",
    "model.vision_encoder.layer2.1.bn2.bias":"res3.1.conv2.norm.bias",
    "model.vision_encoder.layer2.1.conv3.weight":"res3.1.conv3.weight",
    "model.vision_encoder.layer2.1.bn3.running_mean":"res3.1.conv3.norm.running_mean",
    "model.vision_encoder.layer2.1.bn3.running_var":"res3.1.conv3.norm.running_var",
    "model.vision_encoder.layer2.1.bn3.weight":"res3.1.conv3.norm.weight",
    "model.vision_encoder.layer2.1.bn3.bias":"res3.1.conv3.norm.bias",

    # RES3 Block Layer2
    "model.vision_encoder.layer2.2.conv1.weight":"res3.2.conv1.weight",
    "model.vision_encoder.layer2.2.bn1.running_mean":"res3.2.conv1.norm.running_mean",
    "model.vision_encoder.layer2.2.bn1.running_var":"res3.2.conv1.norm.running_var",
    "model.vision_encoder.layer2.2.bn1.weight":"res3.2.conv1.norm.weight",
    "model.vision_encoder.layer2.2.bn1.bias":"res3.2.conv1.norm.bias",
    "model.vision_encoder.layer2.2.conv2.weight":"res3.2.conv2.weight",
    "model.vision_encoder.layer2.2.bn2.running_mean":"res3.2.conv2.norm.running_mean",
    "model.vision_encoder.layer2.2.bn2.running_var":"res3.2.conv2.norm.running_var",
    "model.vision_encoder.layer2.2.bn2.weight":"res3.2.conv2.norm.weight",
    "model.vision_encoder.layer2.2.bn2.bias":"res3.2.conv2.norm.bias",
    "model.vision_encoder.layer2.2.conv3.weight":"res3.2.conv3.weight",
    "model.vision_encoder.layer2.2.bn3.running_mean":"res3.2.conv3.norm.running_mean",
    "model.vision_encoder.layer2.2.bn3.running_var":"res3.2.conv3.norm.running_var",
    "model.vision_encoder.layer2.2.bn3.weight":"res3.2.conv3.norm.weight",
    "model.vision_encoder.layer2.2.bn3.bias":"res3.2.conv3.norm.bias",

    # RES3 Block Layer3
    "model.vision_encoder.layer2.3.conv1.weight":"res3.3.conv1.weight",
    "model.vision_encoder.layer2.3.bn1.running_mean":"res3.3.conv1.norm.running_mean",
    "model.vision_encoder.layer2.3.bn1.running_var":"res3.3.conv1.norm.running_var",
    "model.vision_encoder.layer2.3.bn1.weight":"res3.3.conv1.norm.weight",
    "model.vision_encoder.layer2.3.bn1.bias":"res3.3.conv1.norm.bias",
    "model.vision_encoder.layer2.3.conv2.weight":"res3.3.conv2.weight",
    "model.vision_encoder.layer2.3.bn2.running_mean":"res3.3.conv2.norm.running_mean",
    "model.vision_encoder.layer2.3.bn2.running_var":"res3.3.conv2.norm.running_var",
    "model.vision_encoder.layer2.3.bn2.weight":"res3.3.conv2.norm.weight",
    "model.vision_encoder.layer2.3.bn2.bias":"res3.3.conv2.norm.bias",
    "model.vision_encoder.layer2.3.conv3.weight":"res3.3.conv3.weight",
    "model.vision_encoder.layer2.3.bn3.running_mean":"res3.3.conv3.norm.running_mean",
    "model.vision_encoder.layer2.3.bn3.running_var":"res3.3.conv3.norm.running_var",
    "model.vision_encoder.layer2.3.bn3.weight":"res3.3.conv3.norm.weight",
    "model.vision_encoder.layer2.3.bn3.bias":"res3.3.conv3.norm.bias",

    # RES4 Block Layer0
    "model.vision_encoder.layer3.0.conv1.weight":"res4.0.conv1.weight",
    "model.vision_encoder.layer3.0.bn1.running_mean":"res4.0.conv1.norm.running_mean",
    "model.vision_encoder.layer3.0.bn1.running_var":"res4.0.conv1.norm.running_var",
    "model.vision_encoder.layer3.0.bn1.weight":"res4.0.conv1.norm.weight",
    "model.vision_encoder.layer3.0.bn1.bias":"res4.0.conv1.norm.bias",
    "model.vision_encoder.layer3.0.conv2.weight":"res4.0.conv2.weight",
    "model.vision_encoder.layer3.0.bn2.running_mean":"res4.0.conv2.norm.running_mean",
    "model.vision_encoder.layer3.0.bn2.running_var":"res4.0.conv2.norm.running_var",
    "model.vision_encoder.layer3.0.bn2.weight":"res4.0.conv2.norm.weight",
    "model.vision_encoder.layer3.0.bn2.bias":"res4.0.conv2.norm.bias",
    "model.vision_encoder.layer3.0.conv3.weight":"res4.0.conv3.weight",
    "model.vision_encoder.layer3.0.bn3.running_mean":"res4.0.conv3.norm.running_mean",
    "model.vision_encoder.layer3.0.bn3.running_var":"res4.0.conv3.norm.running_var",
    "model.vision_encoder.layer3.0.bn3.weight":"res4.0.conv3.norm.weight",
    "model.vision_encoder.layer3.0.bn3.bias":"res4.0.conv3.norm.bias",
    "model.vision_encoder.layer3.0.downsample.0.weight":"res4.0.shortcut.weight",
    "model.vision_encoder.layer3.0.downsample.1.running_mean":"res4.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer3.0.downsample.1.running_var":"res4.0.shortcut.norm.running_var",
    "model.vision_encoder.layer3.0.downsample.1.weight":"res4.0.shortcut.norm.weight",
    "model.vision_encoder.layer3.0.downsample.1.bias":"res4.0.shortcut.norm.bias",

    # RES4 Block Layer1
    "model.vision_encoder.layer3.1.conv1.weight":"res4.1.conv1.weight",
    "model.vision_encoder.layer3.1.bn1.running_mean":"res4.1.conv1.norm.running_mean",
    "model.vision_encoder.layer3.1.bn1.running_var":"res4.1.conv1.norm.running_var",
    "model.vision_encoder.layer3.1.bn1.weight":"res4.1.conv1.norm.weight",
    "model.vision_encoder.layer3.1.bn1.bias":"res4.1.conv1.norm.bias",
    "model.vision_encoder.layer3.1.conv2.weight":"res4.1.conv2.weight",
    "model.vision_encoder.layer3.1.bn2.running_mean":"res4.1.conv2.norm.running_mean",
    "model.vision_encoder.layer3.1.bn2.running_var":"res4.1.conv2.norm.running_var",
    "model.vision_encoder.layer3.1.bn2.weight":"res4.1.conv2.norm.weight",
    "model.vision_encoder.layer3.1.bn2.bias":"res4.1.conv2.norm.bias",
    "model.vision_encoder.layer3.1.conv3.weight":"res4.1.conv3.weight",
    "model.vision_encoder.layer3.1.bn3.running_mean":"res4.1.conv3.norm.running_mean",
    "model.vision_encoder.layer3.1.bn3.running_var":"res4.1.conv3.norm.running_var",
    "model.vision_encoder.layer3.1.bn3.weight":"res4.1.conv3.norm.weight",
    "model.vision_encoder.layer3.1.bn3.bias":"res4.1.conv3.norm.bias",

    # RES4 Block Layer2
    "model.vision_encoder.layer3.2.conv1.weight":"res4.2.conv1.weight",
    "model.vision_encoder.layer3.2.bn1.running_mean":"res4.2.conv1.norm.running_mean",
    "model.vision_encoder.layer3.2.bn1.running_var":"res4.2.conv1.norm.running_var",
    "model.vision_encoder.layer3.2.bn1.weight":"res4.2.conv1.norm.weight",
    "model.vision_encoder.layer3.2.bn1.bias":"res4.2.conv1.norm.bias",
    "model.vision_encoder.layer3.2.conv2.weight":"res4.2.conv2.weight",
    "model.vision_encoder.layer3.2.bn2.running_mean":"res4.2.conv2.norm.running_mean",
    "model.vision_encoder.layer3.2.bn2.running_var":"res4.2.conv2.norm.running_var",
    "model.vision_encoder.layer3.2.bn2.weight":"res4.2.conv2.norm.weight",
    "model.vision_encoder.layer3.2.bn2.bias":"res4.2.conv2.norm.bias",
    "model.vision_encoder.layer3.2.conv3.weight":"res4.2.conv3.weight",
    "model.vision_encoder.layer3.2.bn3.running_mean":"res4.2.conv3.norm.running_mean",
    "model.vision_encoder.layer3.2.bn3.running_var":"res4.2.conv3.norm.running_var",
    "model.vision_encoder.layer3.2.bn3.weight":"res4.2.conv3.norm.weight",
    "model.vision_encoder.layer3.2.bn3.bias":"res4.2.conv3.norm.bias",

    # RES4 Block Layer3
    "model.vision_encoder.layer3.3.conv1.weight":"res4.3.conv1.weight",
    "model.vision_encoder.layer3.3.bn1.running_mean":"res4.3.conv1.norm.running_mean",
    "model.vision_encoder.layer3.3.bn1.running_var":"res4.3.conv1.norm.running_var",
    "model.vision_encoder.layer3.3.bn1.weight":"res4.3.conv1.norm.weight",
    "model.vision_encoder.layer3.3.bn1.bias":"res4.3.conv1.norm.bias",
    "model.vision_encoder.layer3.3.conv2.weight":"res4.3.conv2.weight",
    "model.vision_encoder.layer3.3.bn2.running_mean":"res4.3.conv2.norm.running_mean",
    "model.vision_encoder.layer3.3.bn2.running_var":"res4.3.conv2.norm.running_var",
    "model.vision_encoder.layer3.3.bn2.weight":"res4.3.conv2.norm.weight",
    "model.vision_encoder.layer3.3.bn2.bias":"res4.3.conv2.norm.bias",
    "model.vision_encoder.layer3.3.conv3.weight":"res4.3.conv3.weight",
    "model.vision_encoder.layer3.3.bn3.running_mean":"res4.3.conv3.norm.running_mean",
    "model.vision_encoder.layer3.3.bn3.running_var":"res4.3.conv3.norm.running_var",
    "model.vision_encoder.layer3.3.bn3.weight":"res4.3.conv3.norm.weight",
    "model.vision_encoder.layer3.3.bn3.bias":"res4.3.conv3.norm.bias",

    # RES4 Block Layer4
    "model.vision_encoder.layer3.4.conv1.weight":"res4.4.conv1.weight",
    "model.vision_encoder.layer3.4.bn1.running_mean":"res4.4.conv1.norm.running_mean",
    "model.vision_encoder.layer3.4.bn1.running_var":"res4.4.conv1.norm.running_var",
    "model.vision_encoder.layer3.4.bn1.weight":"res4.4.conv1.norm.weight",
    "model.vision_encoder.layer3.4.bn1.bias":"res4.4.conv1.norm.bias",
    "model.vision_encoder.layer3.4.conv2.weight":"res4.4.conv2.weight",
    "model.vision_encoder.layer3.4.bn2.running_mean":"res4.4.conv2.norm.running_mean",
    "model.vision_encoder.layer3.4.bn2.running_var":"res4.4.conv2.norm.running_var",
    "model.vision_encoder.layer3.4.bn2.weight":"res4.4.conv2.norm.weight",
    "model.vision_encoder.layer3.4.bn2.bias":"res4.4.conv2.norm.bias",
    "model.vision_encoder.layer3.4.conv3.weight":"res4.4.conv3.weight",
    "model.vision_encoder.layer3.4.bn3.running_mean":"res4.4.conv3.norm.running_mean",
    "model.vision_encoder.layer3.4.bn3.running_var":"res4.4.conv3.norm.running_var",
    "model.vision_encoder.layer3.4.bn3.weight":"res4.4.conv3.norm.weight",
    "model.vision_encoder.layer3.4.bn3.bias":"res4.4.conv3.norm.bias",

    # RES4 Block Layer5
    "model.vision_encoder.layer3.5.conv1.weight":"res4.5.conv1.weight",
    "model.vision_encoder.layer3.5.bn1.running_mean":"res4.5.conv1.norm.running_mean",
    "model.vision_encoder.layer3.5.bn1.running_var":"res4.5.conv1.norm.running_var",
    "model.vision_encoder.layer3.5.bn1.weight":"res4.5.conv1.norm.weight",
    "model.vision_encoder.layer3.5.bn1.bias":"res4.5.conv1.norm.bias",
    "model.vision_encoder.layer3.5.conv2.weight":"res4.5.conv2.weight",
    "model.vision_encoder.layer3.5.bn2.running_mean":"res4.5.conv2.norm.running_mean",
    "model.vision_encoder.layer3.5.bn2.running_var":"res4.5.conv2.norm.running_var",
    "model.vision_encoder.layer3.5.bn2.weight":"res4.5.conv2.norm.weight",
    "model.vision_encoder.layer3.5.bn2.bias":"res4.5.conv2.norm.bias",
    "model.vision_encoder.layer3.5.conv3.weight":"res4.5.conv3.weight",
    "model.vision_encoder.layer3.5.bn3.running_mean":"res4.5.conv3.norm.running_mean",
    "model.vision_encoder.layer3.5.bn3.running_var":"res4.5.conv3.norm.running_var",
    "model.vision_encoder.layer3.5.bn3.weight":"res4.5.conv3.norm.weight",
    "model.vision_encoder.layer3.5.bn3.bias":"res4.5.conv3.norm.bias",

    # RES5 Block Layer0
    "model.vision_encoder.layer4.0.conv1.weight":"res5.0.conv1.weight",
    "model.vision_encoder.layer4.0.bn1.running_mean":"res5.0.conv1.norm.running_mean",
    "model.vision_encoder.layer4.0.bn1.running_var":"res5.0.conv1.norm.running_var",
    "model.vision_encoder.layer4.0.bn1.weight":"res5.0.conv1.norm.weight",
    "model.vision_encoder.layer4.0.bn1.bias":"res5.0.conv1.norm.bias",
    "model.vision_encoder.layer4.0.conv2.weight":"res5.0.conv2.weight",
    "model.vision_encoder.layer4.0.bn2.running_mean":"res5.0.conv2.norm.running_mean",
    "model.vision_encoder.layer4.0.bn2.running_var":"res5.0.conv2.norm.running_var",
    "model.vision_encoder.layer4.0.bn2.weight":"res5.0.conv2.norm.weight",
    "model.vision_encoder.layer4.0.bn2.bias":"res5.0.conv2.norm.bias",
    "model.vision_encoder.layer4.0.conv3.weight":"res5.0.conv3.weight",
    "model.vision_encoder.layer4.0.bn3.running_mean":"res5.0.conv3.norm.running_mean",
    "model.vision_encoder.layer4.0.bn3.running_var":"res5.0.conv3.norm.running_var",
    "model.vision_encoder.layer4.0.bn3.weight":"res5.0.conv3.norm.weight",
    "model.vision_encoder.layer4.0.bn3.bias":"res5.0.conv3.norm.bias",
    "model.vision_encoder.layer4.0.downsample.0.weight":"res5.0.shortcut.weight",
    "model.vision_encoder.layer4.0.downsample.1.running_mean":"res5.0.shortcut.norm.running_mean",
    "model.vision_encoder.layer4.0.downsample.1.running_var":"res5.0.shortcut.norm.running_var",
    "model.vision_encoder.layer4.0.downsample.1.weight":"res5.0.shortcut.norm.weight",
    "model.vision_encoder.layer4.0.downsample.1.bias":"res5.0.shortcut.norm.bias",

    # RES5 Block Layer1
    "model.vision_encoder.layer4.1.conv1.weight":"res5.1.conv1.weight",
    "model.vision_encoder.layer4.1.bn1.running_mean":"res5.1.conv1.norm.running_mean",
    "model.vision_encoder.layer4.1.bn1.running_var":"res5.1.conv1.norm.running_var",
    "model.vision_encoder.layer4.1.bn1.weight":"res5.1.conv1.norm.weight",
    "model.vision_encoder.layer4.1.bn1.bias":"res5.1.conv1.norm.bias",
    "model.vision_encoder.layer4.1.conv2.weight":"res5.1.conv2.weight",
    "model.vision_encoder.layer4.1.bn2.running_mean":"res5.1.conv2.norm.running_mean",
    "model.vision_encoder.layer4.1.bn2.running_var":"res5.1.conv2.norm.running_var",
    "model.vision_encoder.layer4.1.bn2.weight":"res5.1.conv2.norm.weight",
    "model.vision_encoder.layer4.1.bn2.bias":"res5.1.conv2.norm.bias",
    "model.vision_encoder.layer4.1.conv3.weight":"res5.1.conv3.weight",
    "model.vision_encoder.layer4.1.bn3.running_mean":"res5.1.conv3.norm.running_mean",
    "model.vision_encoder.layer4.1.bn3.running_var":"res5.1.conv3.norm.running_var",
    "model.vision_encoder.layer4.1.bn3.weight":"res5.1.conv3.norm.weight",
    "model.vision_encoder.layer4.1.bn3.bias":"res5.1.conv3.norm.bias",

    # RES5 Block Layer2
    "model.vision_encoder.layer4.2.conv1.weight":"res5.2.conv1.weight",
    "model.vision_encoder.layer4.2.bn1.running_mean":"res5.2.conv1.norm.running_mean",
    "model.vision_encoder.layer4.2.bn1.running_var":"res5.2.conv1.norm.running_var",
    "model.vision_encoder.layer4.2.bn1.weight":"res5.2.conv1.norm.weight",
    "model.vision_encoder.layer4.2.bn1.bias":"res5.2.conv1.norm.bias",
    "model.vision_encoder.layer4.2.conv2.weight":"res5.2.conv2.weight",
    "model.vision_encoder.layer4.2.bn2.running_mean":"res5.2.conv2.norm.running_mean",
    "model.vision_encoder.layer4.2.bn2.running_var":"res5.2.conv2.norm.running_var",
    "model.vision_encoder.layer4.2.bn2.weight":"res5.2.conv2.norm.weight",
    "model.vision_encoder.layer4.2.bn2.bias":"res5.2.conv2.norm.bias",
    "model.vision_encoder.layer4.2.conv3.weight":"res5.2.conv3.weight",
    "model.vision_encoder.layer4.2.bn3.running_mean":"res5.2.conv3.norm.running_mean",
    "model.vision_encoder.layer4.2.bn3.running_var":"res5.2.conv3.norm.running_var",
    "model.vision_encoder.layer4.2.bn3.weight":"res5.2.conv3.norm.weight",
    "model.vision_encoder.layer4.2.bn3.bias":"res5.2.conv3.norm.bias",
}

In [8]:
import numpy as np
from safetensors import safe_open
import pickle

mask2formerResnetPklPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/R-50.pkl'
BigEarthResnetPthPath = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/model_S2.safetensors'

# Keep the Bigearth weights in a dictionary with the keys from the Mask2Former model
bigearth_weights = {}
key_conv_stem = ("model.vision_encoder.conv1.weight","stem.conv1.weight")

with safe_open(BigEarthResnetPthPath, framework="pt") as f:
    for key in f.keys():
        if key in dictionary_from_Sentinel_to_Mask2former:
            new_key = dictionary_from_Sentinel_to_Mask2former[key]
            bigearth_weights[new_key] = np.array(f.get_tensor(key))

    value_stem = np.array(f.get_tensor(key_conv_stem[0]))
    value_stem = value_stem[:,[8,4,3,2],:,:]
    print(value_stem.shape)
    bigearth_weights[key_conv_stem[1]] = value_stem

for key in bigearth_weights.keys():
    print(key)
    print(bigearth_weights[key].shape)

# Load the weights from the pickle file
with open(mask2formerResnetPklPath, 'rb') as f:
    weights = pickle.load(f)


# fix stems convolution channel



weights["model"] = bigearth_weights

# Save the updated weights to a new pickle file
new_pkl_path = '/home/pablo.canosa/wip/python_notebooks/ResnetWeightConversion/R-50_S2_correct_stem_bands.pkl'
with open(new_pkl_path, 'wb') as f:
    pickle.dump(weights, f)


(64, 4, 7, 7)
stem.conv1.norm.bias
(64,)
stem.conv1.norm.running_mean
(64,)
stem.conv1.norm.running_var
(64,)
stem.conv1.norm.weight
(64,)
stem.fc.bias
(19,)
stem.fc.weight 
(19, 2048)
res2.0.conv1.norm.bias
(64,)
res2.0.conv1.norm.running_mean
(64,)
res2.0.conv1.norm.running_var
(64,)
res2.0.conv1.norm.weight
(64,)
res2.0.conv2.norm.bias
(64,)
res2.0.conv2.norm.running_mean
(64,)
res2.0.conv2.norm.running_var
(64,)
res2.0.conv2.norm.weight
(64,)
res2.0.conv3.norm.bias
(256,)
res2.0.conv3.norm.running_mean
(256,)
res2.0.conv3.norm.running_var
(256,)
res2.0.conv3.norm.weight
(256,)
res2.0.conv1.weight
(64, 64, 1, 1)
res2.0.conv2.weight
(64, 64, 3, 3)
res2.0.conv3.weight
(256, 64, 1, 1)
res2.0.shortcut.weight
(256, 64, 1, 1)
res2.0.shortcut.norm.bias
(256,)
res2.0.shortcut.norm.running_mean
(256,)
res2.0.shortcut.norm.running_var
(256,)
res2.0.shortcut.norm.weight
(256,)
res2.1.conv1.norm.bias
(64,)
res2.1.conv1.norm.running_mean
(64,)
res2.1.conv1.norm.running_var
(64,)
res2.1.conv1.nor